In [2]:
import json
from sklearn.metrics import r2_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer


In [10]:
with open('controls/CaSiNo.jsonl', 'r') as fin:
    CaSiNo = json.load(fin)
test_set_size = len(CaSiNo)
agent1, agent2, both = 0, 0, 0
for item in CaSiNo:
    if item['response'].split('.')[0] == item['ground_truth'].split('.')[0]:
        agent1+=1
    if item['response'].split('.')[1] == item['ground_truth'].split('.')[1]:
        agent2+=1
    if item['response']==item['ground_truth']:
        both+=1
print(f'Accuracy for predicting priorities:\nAgent1: {agent1/test_set_size} percents\nAgent2: {agent2/test_set_size} percents\nBoth Agents: {both/test_set_size} percents')

Accuracy for predicting priorities:
Agent1: 0.48 percents
Agent2: 0.54 percents
Both Agents: 0.25 percents


In [16]:
with open('controls/CRAIGSLISTBARGAIN.jsonl', 'r') as fin:
    CRAIGSLISTBARGAIN = json.load(fin)
buyer_price_actual, buyer_price_predicted, seller_price_actual, seller_price_predicted = [], [], [], []
for item in CRAIGSLISTBARGAIN:
    buyer_price_actual.append(int(item['ground_truth'].split(' ')[-2].replace(',', '')))
    buyer_price_predicted.append(int(item['response'].split(' ')[-2].replace(',', '')))
    seller_price_actual.append(int(item['ground_truth'].split(' ')[7].strip(',').replace(',', '')))
    seller_price_predicted.append(int(item['response'].split(' ')[7].strip(',').replace(',', '')))
print(f'R^2 Score for predicted buyer\'s price is {r2_score(buyer_price_actual, buyer_price_predicted)}\nR^2 Score for predicted seller\'s price is {r2_score(seller_price_actual, seller_price_predicted)}')

R^2 Score for predicted buyer's price is 0.9162770767996156
R^2 Score for predicted seller's price is 0.9470176479894197


In [3]:
with open('controls/NegotiationToM.jsonl', 'r') as fin:
    NegotiationToM = json.load(fin)
desire_actual, desire_predicted, belief_actual, belief_predicted, a1_intent_actual, \
intent_actual, intent_predicted, \
a1_intent_predicted, a2_intent_actual, a2_intent_predicted = [], [], [], [], [], [], [], [], [], []

for item in NegotiationToM:

    temp = item['ground_truth'].split('Desire ')
    temp = [label.split(':')[1].split(',')[0].strip() for label in temp[1:]]
    desire_actual.append(temp)
    temp = item['response'].split('Desire ')
    try:
        temp = [label.split(':')[1].split(',')[0].strip() for label in temp[1:]]
    except:
        temp = ['_' for i in range(6)]
    desire_predicted.append(temp)
    
    temp = item['ground_truth'].split('Belief ')
    temp = [label.split(':')[1].replace('.', ',').split(',')[0].strip() for label in temp[1:]]
    belief_actual.append(temp)
    temp = item['response'].split('Belief ')
    try:
        temp = [label.split(':')[1].replace('.', ',').split(',')[0].strip() for label in temp[1:]]
    except:
        temp = ['_' for i in range(6)]
    if len(temp)!=6:
        temp = ['_' for i in range(6)]
    belief_predicted.append(temp)
    
    a1_intent_actual.append(item['ground_truth'].split('[')[1].split(']')[0].split(','))
    try:
        a1_intent_predicted.append(item['response'].split('[')[1].split(']')[0].split(','))
    except:
        a1_intent_predicted.append([])
    if len(temp)!=6:
        temp = ['_' for i in range(6)]
    
    a2_intent_actual.append(item['ground_truth'].split('[')[2].split(']')[0].split(','))
    try:
        a2_intent_predicted.append(item['response'].split('[')[2].split(']')[0].split(','))
    except:
        a2_intent_predicted.append([])

intent_actual = [a1+a2 for a1, a2 in zip(a1_intent_actual, a2_intent_actual)]
intent_predicted = [a1+a2 for a1, a2 in zip(a1_intent_predicted, a2_intent_predicted)]

# print(a1_intent_actual)
# print(a1_intent_predicted)
# print(a2_intent_actual)
# print(a2_intent_predicted)
# print(desire_actual)
# print(desire_predicted)
# print(belief_actual)
# print(belief_predicted)
mlb = MultiLabelBinarizer()
y_true_binary = mlb.fit_transform(intent_actual)
y_pred_binary = mlb.transform(intent_predicted)

# Calculate F1 score
f1_micro = f1_score(y_true_binary, y_pred_binary, average='micro')
f1_macro = f1_score(y_true_binary, y_pred_binary, average='macro')

print(f"F1 Score (Micro): {f1_micro}")
print(f"F1 Score (Macro): {f1_macro}")

test_set_size = len(NegotiationToM)
desire_accuracy = 0
for a, p in zip(desire_predicted, desire_actual):
    if a==p:
        desire_accuracy+=1
print(desire_accuracy/test_set_size)

F1 Score (Micro): 0.6426656453466105
F1 Score (Macro): 0.4827246681902486
0.3558368495077356


In [34]:
from sklearn.metrics import f1_score
f1_score(['Not Given', 'Not Given', 'Not Given', 'Not Given', 'Not Given', 'Not Given'], )

The intent of the Agent 1 is [Build-Rapport,Describe-Need] and the intent of the Agent 2 is [Build-Rapport] Regarding the Agent 1, Desire High: Food, Desire Medium: Not Given, Desire Low: Not Given,  Belief High: Not Given, Belief Medium: Not Given, Belief Low: Not Given. Regarding the Agent 2, Desire High: Not Given, Desire Medium: Not Given, Desire Low: Not Given,  Belief High: Food, Belief Medium: Not Given, Belief Low: Not Given.
The intent of the Agent 1 is [Build-Rapport,Describe-Need] and the intent of the Agent 2 is [Describe-Need] Regarding the Agent 1, Desire High: Not Given, Desire Medium: Not Given, Desire Low: Firewood,  Belief High: Food, Belief Medium: Not Given, Belief Low: Not Given. Regarding the Agent 2, Desire High: Food, Desire Medium: Not Given, Desire Low: Not Given,


In [27]:
for item in NegotiationToM[0]['response'].split('Belief '):
    print(item)

The intent of the Agent 1 is [Build-Rapport] and the intent of the Agent 2 is [Build-Rapport] Regarding the Agent 1, Desire High: Not Given, Desire Medium: Not Given, Desire Low: Not Given,  
High: Not Given, 
Medium: Not Given, 
Low: Not Given. Regarding the Agent 2, Desire High: Not Given, Desire Medium: Not Given, Desire Low: Not Given,


In [15]:
for item in NegotiationToM[0]['response'].split('Desire '):
    print(item)

The intent of the Agent 1 is [Build-Rapport] and the intent of the Agent 2 is [Build-Rapport] Regarding the Agent 1, 
High: Not Given, 
Medium: Not Given, 
Low: Not Given,  Belief High: Not Given, Belief Medium: Not Given, Belief Low: Not Given. Regarding the Agent 2, 
High: Not Given, 
Medium: Not Given, 
Low: Not Given,


In [28]:
def get_NegotiationToM_text(train_config, tokenizer, train=True):
    data_path = train_config.train_qa if train else train_config.eval_qa
    with open(data_path, 'rb') as fin:
        data = json.load(fin)
    print(data[0])
    read_prompts, QAs = [], []
    for item in data:
        temp = []
        for uttrance in item['dialogue']:
            if uttrance.split(': ')[0] == 'agent_2':
                temp.append({"role": "Agent 2", "content": uttrance.split(': ')[1]})
            elif uttrance.split(': ')[0] == 'agent_1':
                temp.append({"role": "Agent 1", "content": uttrance.split(': ')[1]})
        read_prompts.append(temp)
        
        question = "what is the intent of each agent for the last utterances? What are the beliefs and desires of each agent?"
        agent1_intent = item['utterance1_intent'] if item['utterance1_agent']=='agent_1' else item['utterance2_intent']
        agent2_intent = item['utterance1_intent'] if item['utterance1_agent']=='agent_2' else item['utterance2_intent']
        
        answer = f"The intent of the Agent 1 is [{agent1_intent}] and the intent of the Agent 2 is [{agent2_intent}]"
        answer += f' Regarding the Agent 1, Desire High: {item["agent1_desire_high"]}, Desire Medium: {item["agent1_desire_medium"]}, Desire Low: {item["agent1_desire_low"]},  Belief High: {item["agent1_belief_high"]}, Belief Medium: {item["agent1_belief_medium"]}, Belief Low: {item["agent1_belief_low"]}.' 
        answer += f' Regarding the Agent 2, Desire High: {item["agent2_desire_high"]}, Desire Medium: {item["agent2_desire_medium"]}, Desire Low: {item["agent2_desire_low"]},  Belief High: {item["agent2_belief_high"]}, Belief Medium: {item["agent2_belief_medium"]}, Belief Low: {item["agent2_belief_low"]}.' 
        QAs.append([
                {"role": "user", "content": question},
                {"role": "assistant", "content": answer}
                ])

    assert len(QAs)==len(read_prompts)
    return read_prompts, QAs
    


In [29]:
read_prompts, QAs = get_NegotiationToM_text(None, None, None)
len(read_prompts), len(QAs)

{'dialogue_id': '2-0', 'dialogue': ['agent_2: Hey there!', 'agent_1: Hi! How are you?! You excited for your camping trip??! I sure am ready to go on mine!'], 'agent1_desire': {'Medium': 'Food', 'High': 'Water', 'Low': 'Firewood'}, 'agent2_desire': {'Low': 'Food', 'High': 'Water', 'Medium': 'Firewood'}, 'intents': ['No-Intention', 'Build-Rapport', 'Build-Rapport,Describe-Need,Discover-Preference', 'Build-Rapport,Describe-Need', 'Show-Empathy,Undermine-Requirements', 'Discover-Preference,Promote-Coordination', 'Promote-Coordination', 'Describe-Need,Undermine-Requirements', 'Describe-Need,Promote-Coordination', 'No-Intention'], 'utterance1_intent': 'No-Intention', 'utterance2_intent': 'Build-Rapport', 'utterance1_agent': 'agent_2', 'utterance2_agent': 'agent_1', 'agent1_desire_high': 'Not Given', 'agent1_desire_medium': 'Not Given', 'agent1_desire_low': 'Not Given', 'agent1_belief_high': 'Not Given', 'agent1_belief_medium': 'Not Given', 'agent1_belief_low': 'Not Given', 'agent2_desire_hig

(1335, 1335)

In [27]:
read_prompts[0], QAs[0]

([{'role': 'Agent 2', 'content': 'Hey there!'},
  {'role': 'Agent 1',
   'content': 'Hi! How are you?! You excited for your camping trip??! I sure am ready to go on mine!'}],
 [{'role': 'user',
   'content': 'what is the intent of each agent for the last utterances? What are the beliefs and desires of each agent?'},
  {'role': 'assistant',
   'content': 'The intent of the Agent 1 is [Build-Rapport] and the intent of the Agent 2 is [No-Intention] Regarding the Agent 1, Desire High: Not Given, Desire Medium: Not Given, Desire Low: Not Given,  Belief High: Not Given, Belief Medium: Not Given, Belief Low: Not Given. Regarding the Agent 2, Desire High: Not Given, Desire Medium: Not Given, Desire Low: Not Given,  Belief High: Not Given, Belief Medium: Not Given, Belief Low: Not Given.'}])

In [8]:
import re

def extract_desires_and_beliefs(text):
    """
    Extract Desire and Belief values for Agent 1 and Agent 2 from the given text.

    Parameters:
    text (str): Input string containing the intents and values for agents.

    Returns:
    dict: A dictionary containing extracted values for each agent.
    """
    # Regex pattern to extract Desire and Belief for each agent
    pattern = (
        r"Regarding the Agent (?P<agent>\d), "
        r"Desire High: (?P<desire_high>[\w\s]+), "
        r"Desire Medium: (?P<desire_medium>[\w\s]+), "
        r"Desire Low: (?P<desire_low>[\w\s]+), "
        r"Belief High: (?P<belief_high>[\w\s]+), "
        r"Belief Medium: (?P<belief_medium>[\w\s]+), "
        r"Belief Low: (?P<belief_low>[\w\s]+)\."
    )

    matches = re.finditer(pattern, text)

    # Parse the matches into a dictionary
    result = {}
    for match in matches:
        agent = f"Agent {match.group('agent')}"
        result[agent] = {
            "Desire": {
                "High": match.group('desire_high').strip(),
                "Medium": match.group('desire_medium').strip(),
                "Low": match.group('desire_low').strip()
            },
            "Belief": {
                "High": match.group('belief_high').strip(),
                "Medium": match.group('belief_medium').strip(),
                "Low": match.group('belief_low').strip()
            }
        }

    return result


In [9]:
extract_desires_and_beliefs(NegotiationToM[0]['ground_truth'])

{}